# Skeleton-of-Thought (SoT) Prompting

This notebook demonstrates **Skeleton-of-Thought (SoT)**, a technique designed to accelerate high-quality content generation and improve structure.

**The Concept:**
When humans write long essays, we usually outline first. **SoT** forces the LLM to do the same:
1.  **Skeleton Stage**: The model generates a concise outline (the "Skeleton") of the answer.
2.  **Point Expansion**: We treat each point in the outline as a separate sub-task and ask the model to expand on it.
3.  **Combination**: We merge the expanded sections into a final document.

**Key Mechanics:**
* **Structure**: By defining the skeleton first, the final output is more coherent and less likely to ramble.
* **Speed**: In production systems, the "Point Expansion" phase can be run in **parallel** (concurrent API calls), drastically reducing the time needed to generate long reports.

In [ ]:
%pip install openai python-dotenv --quiet

### 1. Setup and Authorization

We start by importing the necessary libraries and loading your OpenAI API key.

In [ ]:
from openai import OpenAI
import os
import time
from dotenv import load_dotenv

load_dotenv()

api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    api_key = input("Paste your OpenAI API key: ").strip()

# Model configuration - can be overridden via environment variable
MODEL = os.getenv("OPENAI_MODEL", "gpt-4o-mini")

client = OpenAI(api_key=api_key)
print(f"OpenAI client ready! Using model: {MODEL}")

### 2. Stage 1: Generating the Skeleton

In this step, we ask the model to create a high-level structure. We explicitly ask for a **numbered list** to make parsing easier later.

**Note:** We use a strict prompt to ensure it *only* gives us the outline, not the full text yet.

In [ ]:
def generate_skeleton(query):
    print(f"Creating skeleton for: '{query}'...\n")
    
    try:
        response = client.chat.completions.create(
            model=MODEL,
            messages=[{
                "role": "user", 
                "content": f"""
You are an expert outline writer.
Create a concise, detailed numbered skeleton outline (8–12 points) for answering this request:

Request: {query}

Return ONLY the numbered list. No introduction, no conclusion, no full text.
"""
            }],
            temperature=0.3 # Low temp for a logical, standard structure
        )
        
        skeleton_text = response.choices[0].message.content.strip()
        print(f"--- SKELETON ---\n{skeleton_text}\n----------------")
        
        # Parse the skeleton into a list of points
        # We assume valid points start with a digit (e.g., "1. Introduction")
        points = [
            line.strip() 
            for line in skeleton_text.split("\n") 
            if line.strip() and line.strip()[0].isdigit()
        ]
        
        return points
        
    except Exception as e:
        print(f"Error generating skeleton: {e}")
        return []

### 3. Stage 2: Parallel Expansion (The "Flesh")

Now we iterate through the skeleton. For each point, we send a specific prompt to the model: *"Write the content for this specific section."*

**Why is this powerful?**
* **Focus**: The model answers one specific sub-topic at a time, leading to more detailed and accurate content.
* **Context**: We pass the original `query` along with the `point` so the model knows the overall context.

In [ ]:
def expand_points(points, original_query):
    print(f"\nExpanding {len(points)} sections...\n")
    full_document = []
    
    for i, point in enumerate(points, 1):
        print(f"  > Writing section {i}/{len(points)}: {point[:50]}...", end="\r")
        
        try:
            response = client.chat.completions.create(
                model=MODEL,
                messages=[{
                    "role": "user", 
                    "content": f"""
You are writing one section of a larger document.

Topic: {original_query}
Current Section Goal: {point}

Write a detailed, comprehensive paragraph(s) for ONLY this section. 
Do not write an intro or conclusion for the whole document, just this part.
"""
                }],
                temperature=0.7 # Higher temp for creative writing flow
            )
            
            content = response.choices[0].message.content.strip()
            
            # Format: Header + Content
            section_text = f"## {point}\n{content}"
            full_document.append(section_text)
            
        except Exception as e:
            print(f"\n  Error expanding section {i}: {e}")
            full_document.append(f"## {point}\n[Error: Could not expand this section]")
        
        # Brief pause to respect rate limits (optional)
        time.sleep(0.2)
        
    print(f"  Expansion complete!                        ")
    return "\n\n".join(full_document)

### 4. Running the SoT Process

Let's try it on a topic that usually requires a long, structured answer, like a blog post or a guide.

In [ ]:
user_topic = input("Enter a topic (e.g., 'A guide to growing tomatoes' or 'The history of the internet'): ")

if user_topic:
    # 1. Get the Outline
    skeleton_points = generate_skeleton(user_topic)
    
    if skeleton_points:
        # 2. Fill in the Content
        final_doc = expand_points(skeleton_points, user_topic)
        
        # 3. Display Result
        print("\n" + "="*60)
        print(f"# {user_topic.upper()}")
        print("="*60)
        print(final_doc)